# This is a basic RNN model used to test dataset preperation and model evaluation

## Imports

In [ ]:
import torch.nn as nn
import torch
import full_iri_dataset_generator as iri
from training_loop import train_model

## Constants

- `SEQUENCE_LENGTH` is the number of historical measurements before the target element to provide to the model
- `NUM_FREATURES_PER_SAMPLE` is how many details each measurement has. `IRI-only` has 3: left_iri, right_iri, and time_since_first_measurement
- `NUM_LAYERS` is the number of RNN layers to use

In [ ]:
SEQUENCE_LENGTH = 10
NUM_FEATURES_PER_SAMPLE = 14
NUM_LAYERS = 5

## Dataset Preperation

Load train and test datasets

In [ ]:
train, test = iri.load_iri_datasets(path="../training_data/final_data.parquet",
                                    construction_path="../training_data/construction_data.parquet",
                                    seq_length=SEQUENCE_LENGTH)
train2, test2 = iri.load_iri_datasets(path="../training_data/final_data.parquet",
                                    construction_path="../training_data/construction_data.parquet",
                                    seq_length=SEQUENCE_LENGTH, seed=127)
train3, test3 = iri.load_iri_datasets(path="../training_data/final_data.parquet",
                                    construction_path="../training_data/construction_data.parquet",
                                    seq_length=SEQUENCE_LENGTH, seed=83)

## Model Definition

Here a basic RNN classifier model is defined.

1. Data is flattened
2. RNN layers process data and modify hidden state
3. final layer maps hidden state to predicted left and right wheel IRIs
4. outputs are scaled using a logsoftmax function

In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size=SEQUENCE_LENGTH,
                          hidden_size=SEQUENCE_LENGTH * NUM_FEATURES_PER_SAMPLE,
                          num_layers=NUM_LAYERS,
                          batch_first=True)
        self.final = nn.Linear(SEQUENCE_LENGTH * NUM_FEATURES_PER_SAMPLE, 2)
        
    def forward(self, x):
        #transpose x
        # x = x.transpose(1,2)
        h0 = torch.zeros(NUM_LAYERS,
                         x.size(0),
                         SEQUENCE_LENGTH * NUM_FEATURES_PER_SAMPLE).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.final(out[:, -1, :])
        return out
    
class RNN_T(nn.Module):
    def __init__(self):
        super(RNN_T, self).__init__()
        self.rnn = nn.RNN(input_size=NUM_FEATURES_PER_SAMPLE,
                          hidden_size=SEQUENCE_LENGTH * NUM_FEATURES_PER_SAMPLE,
                          num_layers=NUM_LAYERS,
                          batch_first=True)
        self.final = nn.Linear(SEQUENCE_LENGTH * NUM_FEATURES_PER_SAMPLE, 2)
        
    def forward(self, x):
        #transpose x
        x = x.transpose(1,2)
        h0 = torch.zeros(NUM_LAYERS,
                         x.size(0),
                         SEQUENCE_LENGTH * NUM_FEATURES_PER_SAMPLE).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.final(out[:, -1, :])
        return out

## Training

In [ ]:
model1 = RNN()
model2 = RNN()
model3 = RNN()
loss = nn.MSELoss()
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.0005)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.0005)
optimizer3 = torch.optim.Adam(model3.parameters(), lr=0.0005)

improved = train_model(model1, train, test, loss, optimizer1, epochs=100, test_every_n=1, batch_size=512)
improved2 = train_model(model2, train2, test2, loss, optimizer2, epochs=100, test_every_n=1, batch_size=512)
improved3 = train_model(model3, train3, test3, loss, optimizer3, epochs=100, test_every_n=1, batch_size=512)
# write the training data to a file named "norm"


In [ ]:
print(f"ver1: trainloss: {improved['train_losses'][-1]}, testloss: {improved['test_losses'][-1]}, trainr2: {improved['train_r2s'][-1]}, testr2: {improved['test_r2s'][-1]}")
print(f"ver2: trainloss: {improved2['train_losses'][-1]}, testloss: {improved2['test_losses'][-1]}, trainr2: {improved2['train_r2s'][-1]}, testr2: {improved2['test_r2s'][-1]}")
print(f"ver3: trainloss: {improved3['train_losses'][-1]}, testloss: {improved3['test_losses'][-1]}, trainr2: {improved3['train_r2s'][-1]}, testr2: {improved3['test_r2s'][-1]}")

In [ ]:
model_t = RNN_T()
loss_t = nn.MSELoss()
optimizer_t = torch.optim.Adam(model_t.parameters(), lr=0.00001)
lr_scheduler_t = torch.optim.lr_scheduler.StepLR(optimizer_t, step_size=10, gamma=0.75)

traindata = train_model(model_t, train, test, loss_t, optimizer_t, epochs=100, test_every_n=1, batch_size=512)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def scaler(x):
    return (np.sqrt(x) * iri.iri_range) ** 2 
plt.plot(scaler(traindata["train_losses"][1:]), label="Train Loss", color="black", linestyle="--")
plt.plot(scaler(traindata["test_losses"][1:]), label="Test Loss", color="black")
plt.plot(scaler(improved["train_losses"][1:]), label="Train Loss (Transposed Data)", color="#92d050", linestyle="--")
plt.plot(scaler(improved["test_losses"][1:]), label="Test Loss (Transposed Data)", color="#92d050")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.show()

In [ ]:

plt.plot(traindata["train_r2s"][1:], label="Train $R^2$", color="black", linestyle="--")
plt.plot(traindata["test_r2s"][1:], label="Test $R^2$", color="black")
plt.plot(improved["train_r2s"][1:], label="Train $R^2$ (Transposed Data)", color="#92d050", linestyle="--")
plt.plot(improved["test_r2s"][1:], label="Test $R^2$ (Transposed Data)", color="#92d050")
plt.xlabel("Epoch")
plt.ylabel("R2")
plt.legend()
plt.show()

## R^2 Evaluation

In [ ]:
from torcheval.metrics import R2Score
from torch.utils.data import DataLoader

def compute_r2_for(dataset):
    r2 = R2Score()
    train_data = DataLoader(dataset, batch_size=256, shuffle=True)
    for _, data in enumerate(train_data):
        inputs, goal = data[0], data[1]
        outputs = model1(inputs)
        r2.update(goal, outputs)
    return r2.compute()

model1.to("cpu")
model1.eval()
with torch.no_grad():
    train_r2 = compute_r2_for(train)
    print(f"R^2 for training data: {train_r2}")
    test_r2 = compute_r2_for(test)
    print(f"R^2 for testing data: {test_r2}")

## Accuracy

In [ ]:
from torcheval.metrics import MeanSquaredError

def compute_mse_for(dataset):
    mse = MeanSquaredError()
    train_data = DataLoader(dataset, batch_size=256, shuffle=True)
    for _, data in enumerate(train_data):
        inputs, goal = data[0], data[1]
        outputs = model1(inputs)
        mse.update(outputs, goal)
    return mse.compute()

model1.to("cpu")
model1.eval()
with torch.no_grad():
    train_mse = compute_mse_for(train)
    print(f"MSE for training data: {train_mse}")
    test_mse = compute_mse_for(test)
    print(f"MSE for testing data: {test_mse}")

## Error distribution

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

def compute_errors_for(dataset):
    train_data = DataLoader(dataset, batch_size=256, shuffle=True)
    errors = []
    for _, data in enumerate(train_data):
        inputs, goal = data[0], data[1]
        outputs = np.array(model1(inputs))
        errors.append(np.abs((outputs - np.array(goal)).mean(axis=1)))
    return np.concatenate(errors)

model1.to("cpu")
model1.eval()
with torch.no_grad():
    train_errors = compute_errors_for(train)
    test_errors = compute_errors_for(test)

In [ ]:
plt.hist(train_errors, bins=100, label="train")
plt.show()

In [ ]:
plt.hist(test_errors, bins=100, label="test")
plt.show()

In [ ]:
print("Details of train errors:")
train_errors_df = pd.DataFrame(train_errors)
train_errors_df.describe()

In [ ]:
print("Details of test errors:")
test_errors_df = pd.DataFrame(test_errors)
test_errors_df.describe()

## Classification Evaluation

This was made for testing to compare this to the direct classification model to debug performance issues.

In [ ]:
right = 0
wrong = 0


def increment_category_values(predicted, target):
    global right, wrong

    def get_slot(mean):
        boundries = [1.5, 2.68]
        if mean < (boundries[0] - iri.mean_iri) / iri.iri_range:
            return 0
        elif mean < (boundries[1] - iri.mean_iri) / iri.iri_range:
            return 1
        else:
            return 2
        
    if get_slot(predicted.mean()) == get_slot(target.mean()):
        right += 1
    else:
        wrong += 1

def evaluate_categorical_accuracy(dataset):
    global right, wrong
    right = 0
    wrong = 0
    train_data = DataLoader(dataset, batch_size=256, shuffle=True)
    for _, data in enumerate(train_data):
        inputs, goal = data[0], data[1]
        outputs = model1(inputs)
        for pred, target in zip(outputs, goal):
            increment_category_values(pred, target)
    return right / (right + wrong)

device = device = torch.device("cuda")
model1.eval()
with torch.no_grad():
    train_classification_accuracy = evaluate_categorical_accuracy(train)
    print(f'Classification Accuracy of the network on the train data: {100 * train_classification_accuracy}%')
    train_classification_accuracy = evaluate_categorical_accuracy(test)
    print(f'Classification Accuracy of the network on the test data: {100 * train_classification_accuracy}%')